### Conversational Chain

**Indexing Embeddings**

In [24]:
# Step 1: Load, Chunk, Embed, Store

from langchain_community.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings

# Load documents from a URL
docs = WebBaseLoader("https://docs.smith.langchain.com/user_guide").load()  

# Split documents into chunks
chunk = RecursiveCharacterTextSplitter()
documents = chunk.split_documents(docs)              
document_texts = [doc.page_content for doc in documents]  # Extract text

# Embed document texts
embed = OllamaEmbeddings(model="nomic-embed-text")
embeddings = embed.embed_documents(document_texts)  # Generate embeddings

# Create and save FAISS vector store
vector = FAISS.from_embeddings(list(zip(document_texts, embeddings)), embedding=embed)
vector.save_local("C:/Users/admin/OneDrive/Desktop/RAG'S/vector db/Faiss-index/")  # Store locally

USER_AGENT environment variable not set, consider setting it to identify your requests.


**Retrieval & Generation**

In [25]:
import os
from langchain_community.vectorstores import FAISS
from langchain_ollama import OllamaEmbeddings
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_groq import ChatGroq

# Step 1: Setup retriever and embeddings
embed = OllamaEmbeddings(model="nomic-embed-text")
vector = FAISS.load_local("C:/Users/admin/OneDrive/Desktop/RAG'S/vector db/Faiss-index/", embeddings=embed, allow_dangerous_deserialization=True)
retriever = vector.as_retriever()

# Initialize LLM
os.environ["GROQ_API_KEY"] = "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"
llm_mixtral = ChatGroq(model="mixtral-8x7b-32768")

# Step 2: Contextualize User’s Question (History-Aware Retriever Chain)
retrieval_prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Generate a search query to look up relevant information")
])
retriever_chain = create_history_aware_retriever(llm_mixtral, retriever, retrieval_prompt)

# Step 3: Document Chain to Process Retrieved Context (Answer Generation)
response_prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's query based on the following retrieved context:\n\n{context}"),
    ("user", "{input}")
])
document_chain = create_stuff_documents_chain(llm_mixtral, response_prompt)

# Step 4: Combine the Retrieval and Document Processing Chains into a single retrieval chain
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

# Initialize chat history
chat_history = []

# Function to process queries and update chat history
def ask_question(query):  
    global chat_history  
    response = retrieval_chain.invoke({"chat_history": chat_history, "input": query})
    answer = response['answer']
    chat_history.append(HumanMessage(content=query))
    chat_history.append(AIMessage(content=answer))
    return answer

In [28]:
user_input=ask_question("what is application development?")
print("Reply:", user_input)

Reply: Application development is the process of creating, designing, and implementing software applications for various devices and platforms. It involves various stages such as prototyping, beta testing, and production, where developers use tools and platforms like LangSmith to monitor, test, and improve the performance of their applications. LangSmith provides features such as tracing, evaluation, production monitoring, and automations to support developers in the application development lifecycle.


In [9]:
# Chatbot loop
user_input = input("Enter something (type 'exit' to quit): ")

while user_input != 'exit':  # Loop until 'exit' is typed
    print("Reply:", ask_question(user_input))  # Process user query and show AI's reply
    user_input = input("Enter something (type 'exit' to quit): ")

print("Goodbye!")  # End the conversation


Reply: Based on the provided context, LangSmith is a platform for LLM (large language model) application development, monitoring, and testing. In the Beta Testing phase, it's important to collect human feedback on the application's responses, annotate traces, and add runs to datasets for refining and improving performance. In the Production phase, closely inspecting key data points, growing benchmarking datasets, annotating traces, and drilling down into important data in trace view are essential workflows.

Monitoring charts in LangSmith allow tracking of key metrics over time, and tag and metadata grouping help users mark different versions of their applications and view how they perform side-by-side within each chart, which is useful for A/B testing changes in prompt, model, or retrieval strategy.

Automations allow users to perform actions on traces in near real-time, and threads group traces from a single conversation together, making it easier to track the performance of and anno